<h1>Projet 1: Analysez des données de systèmes éducatifs</h1>

Nordine Oural - 29/12/2022

---

**Table of contents**<a id='toc0_'></a>    
- [Introduction](#toc1_1_)    
- [Import des différents modules](#toc1_2_)    
- [Visualisation des données de départ](#toc1_3_)    
  - [Indicateurs](#toc1_3_1_)    
  - [Pays](#toc1_3_2_)    
- [Paramètres initiaux](#toc1_4_)    
- [Nettoyage des pays](#toc1_5_)    
- [Chargement des données à analyser](#toc1_6_)    
- [Filtrage des données par pays](#toc1_7_)    
- [Filtrage des données par indicateurs](#toc1_8_)    
- [Simplification des données selon les années](#toc1_9_)    
- [Execution au sein de la boucle des années](#toc1_10_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

### <a id='toc1_1_'></a>[Introduction](#toc0_)

Le but de ce Notebook est de reprendre le code du Notebook OURAL_Nordine_1_notebook pour calculer les différents classements par année de 2000 à 2016.

Le code utilisé est donc le même, on a juste imbriquer une partie de celui-ci dans une boucle allant de 2000 à 2017.


### <a id='toc1_2_'></a>[Import des différents modules](#toc0_)
Pour notre étude, nous utiliserons les librairies Pandas, NumPy ainsi que Express et graph_objects de PlotLY

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px, plotly.graph_objects as go
from plotly.subplots import make_subplots

### <a id='toc1_3_'></a>[Visualisation des données de départ](#toc0_)

#### <a id='toc1_3_1_'></a>[Indicateurs](#toc0_)

#### <a id='toc1_3_2_'></a>[Pays](#toc0_)
On importe la liste des pays depuis le fichier EdStatsCountry.csv 

In [2]:
countries = pd.read_csv("assets/EdStatsCountry.csv")

### <a id='toc1_4_'></a>[Paramètres initiaux](#toc0_)
Afin de pouvoir paramétrer facilement notre étude, nous définissons toutes les variables de paramètrage à un seul endroit

In [3]:
''' Seuil minimal pour considérer l'étude d'un pays '''
population_threshold = 2000000

''' mapping code indicateur / label '''
final_indicators = [
    {
        "name": "% PC Population",
        "indicators": "IT.CMP.PCMP.P2"
    },
    {
        "name": "% Internet Population",
        "indicators": "IT.NET.USER.P2"
    },
    {
        "name": "PIB/habitant",
        "indicators": "NY.GDP.PCAP.KD"
    },
    {
        "name": "PIB",
        "indicators": "NY.GDP.MKTP.KD"
    },
    {
        "name": "Population Totale",
        "indicators": "SP.POP.TOTL"
    },
    {
        "name": "Population 1419",
        "indicators": "SP.POP.1419.TO.UN"
    },
    {
        "name": "Population -14",
        "indicators": "SP.POP.0014.TO"
    }
]

A partir des paramètres fournis, on crée la liste des indicateurs à retenir

In [4]:
indicators = list(map(lambda x: x["indicators"], final_indicators))


### <a id='toc1_5_'></a>[Nettoyage des pays](#toc0_)

In [5]:
country_codes = pd.read_csv("https://datahub.io/core/country-codes/r/country-codes.csv")

In [6]:
countries = countries.loc[countries["2-alpha code"].isin(country_codes["ISO3166-1-Alpha-2"])]

Il y avait donc dans le fichier de départ des pays 241 - 216 = 25 lignes qui ne correspondaient pas à des pays réels.

### <a id='toc1_6_'></a>[Chargement des données à analyser](#toc0_)

On charge le fichier EdStatsData.csv

In [7]:
data= pd.read_csv("assets/EdStatsData.csv")

In [8]:
cols = list(data)

### <a id='toc1_7_'></a>[Filtrage des données par pays](#toc0_)
Nous devons exclure du dataframe précedent les pays qui ne sont pas de réels pays

In [9]:
real_country_codes = countries["Country Code"]
data_orig = data.copy()

data = data.loc[data["Country Code"].isin(real_country_codes)]
data.reset_index(drop=True, inplace=True)

### <a id='toc1_8_'></a>[Filtrage des données par indicateurs](#toc0_)

Les indicateurs retenus sont dans la liste "indicators" définie dans le chapitre "Paramètres initiaux". On filtre donc le dataframe pour ne retenir que les individus dont la variable "Indicator Code" est contenue dans cette liste.

In [10]:
data = data.loc[data["Indicator Code"].isin(indicators), :]
data.reset_index(drop=True, inplace=True)

### <a id='toc1_9_'></a>[Simplification des données selon les années](#toc0_)

Nous avons vu que que les colonnes comportaient les valeurs pour différentes années entre 1970 et 2100 mais pas de manière uniforme.
Ici, nous ne prendrons en compte que la dernière valeur passée définie.

Pour cela, nous définissons une fonction get_latest_value qui va, pour un individu donné, partir de l'année en cours et qui va chercher la 1ère année qui contient une valeur. La recherche se fera jusqu'à 1970.

In [11]:
data_orig = data.copy()

In [12]:
def get_latest_value(row, current_year=2016):
    while current_year>=1970: 
        s_current_year = str(current_year)
        # cols a été calculé plus haut et contient les noms de toutes les variables du dataframe
        if s_current_year in cols and not pd.isna(row[s_current_year]):
            return row[s_current_year]
        else:
            current_year -= 1
    return None

def rename_column(t):
    for f_i in final_indicators:
        if f_i["indicators"]==t:
            return f_i["name"]
    return t
    
def weighted(x, cols, w="Population Totale"):
    return pd.Series(np.average(x[cols], weights=x[w], axis=0), cols)

def summed(x, cols):
    return pd.Series(np.sum(x[cols], axis=0), cols)

### <a id='toc1_10_'></a>[Execution au sein de la boucle des années](#toc0_)

on commence d'abord par créer le dossier images si celui-ci n'existe pas

In [13]:
import os
if not os.path.exists('images'):
   os.makedirs('images')

Ensuite, on reprend le code originel, on l'applique dans une boucle pour les années, en s'assurant de passer l'annnée en cours à la fonction get_latest_value

In [14]:
for upto_year in range(2000, 2017):
    data = data_orig.copy()
    data["latest_value"] = data.apply(lambda x: get_latest_value(x, upto_year), axis=1)
    current_year = 2100 # la dernière année du dataframe
    while current_year>=1970: 
        s_current_year = str(current_year)
        if s_current_year in cols:
            del data[s_current_year]
        current_year -= 1

    data2 = pd.DataFrame() #1
    # on boucle sur les différents pays
    for country_code in data["Country Code"].unique(): #2
        '''
        2.1
        '''
        tmp = data.loc[data["Country Code"]==country_code]
        country_dict = {"Country Code": [country_code], "Country Name": [tmp["Country Name"].iloc[0]]} #2.1
        for indicator in indicators:
            country_dict[indicator] = [tmp.loc[tmp["Indicator Code"]==indicator, :]["latest_value"].iloc[0]]

        country_df = pd.DataFrame(country_dict) # 2.2
        data2 = pd.concat([data2, country_df], ignore_index=True) #2.3

    data = data2 #3

    countries2 = countries.loc[:,["Country Code", "Region"]]
    data = data.merge(countries2, on="Country Code")

    data = data.rename(columns=rename_column).reset_index(drop=True)

    data = data.loc[~data["Population Totale"].isna()].reset_index(drop=True)
    data = data.loc[~data["Region"].isna()].reset_index(drop=True)

    data.loc[(data["PIB/habitant"].isna()),"PIB/habitant"] = data.loc[(data["PIB/habitant"].isna()),"PIB"] / data.loc[(data["PIB/habitant"].isna()),"Population Totale"]
    data.loc[(data["PIB/habitant"].isna())]

    data.loc[(data["Population 1419"].isna()),"Population 1419"] = data["Population -14"] * 6 / 14

    data.loc[(data["Population -14"].isna()),"Population -14"] = data["Population 1419"] * 6 / 14

    data = data.loc[data["Population Totale"]>population_threshold].reset_index(drop=True)

    data = data.loc[~(data["% PC Population"].isna() | data["% Internet Population"].isna() | data["PIB/habitant"].isna() |  data["Population 1419"].isna() |  data["Population -14"].isna() )].reset_index(drop=True)

    del data["PIB"]
    data["Population 20+"] = data["Population Totale"] - data["Population -14"] - data["Population 1419"]
    del data["Population 1419"]
    del data["Population -14"]

    data_regions1 = data.groupby("Region").apply(weighted, ["% PC Population", "% Internet Population", "PIB/habitant"]).reset_index()
    data_regions2 = data.groupby("Region").apply(summed, ["Population Totale", "Population 20+"]).reset_index()

    data_regions = data_regions2.merge(data_regions1, on="Region")

    data_regions_normalized = data_regions.copy()
    subjects = list(data_regions)[1:]
    for s in subjects:
        data_regions_normalized[s] = data_regions_normalized[s] / np.max(data_regions_normalized[s])
    subjects.append(subjects[0])

    fig = go.Figure()

    for region in data_regions["Region"]:
        r = data_regions_normalized.loc[data_regions["Region"]==region,].iloc[0].values.tolist()
        r = r[1:]
        r.append(r[0])
        fig.add_trace(go.Scatterpolar(
            r=r,
            theta=subjects,
            fill='toself',
            name=region
        ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )
        ),
        showlegend=True,
        height=800,
        width=1000,
        title=go.layout.Title(
            text=f"Répartition en radar des indicateurs retenus - année {upto_year}<br><sup>Chaque indicateur est rapporté par rapport au max de la variable correspondante</sup>",
            xref="paper",
            x=0
        )
    )
    fig.write_image(f"images/radar_{upto_year}.png")

    data_regions["Population 20-"] = data_regions["Population Totale"] - data_regions["Population 20+"]

    def calcul_indicateur_confiance(x):
        return x["% PC Population"] * x["% Internet Population"] + x["% Internet Population"] * 100 * x["PIB/habitant"] / x["PIB/habitant"].max()

    data_regions["indicateur_confiance_individuel"] = calcul_indicateur_confiance(data_regions)
    #On normalise par rapport aux min et max
    data_regions["indicateur_confiance_individuel"] = (data_regions["indicateur_confiance_individuel"] - data_regions["indicateur_confiance_individuel"].min()) / (data_regions["indicateur_confiance_individuel"].max() - data_regions["indicateur_confiance_individuel"].min())

    data_regions["indicateur_confiance_global"] = round(data_regions["indicateur_confiance_individuel"] * np.sqrt(data_regions["Population 20+"] + 0.25 * data_regions["Population 20-"]))
    #On normalise par rapport aux min et max
    data_regions["indicateur_confiance_global"] = (data_regions["indicateur_confiance_global"] - data_regions["indicateur_confiance_global"].min()) / (data_regions["indicateur_confiance_global"].max() - data_regions["indicateur_confiance_global"].min())

    fig = make_subplots(rows=1, cols=1, specs=[[{'type':'domain'}]])
    #fig.add_trace(go.Pie(labels=data_regions['Region'], values=data_regions['indicateur_confiance_individuel'], name="Par personne"), 1, 1)
    fig.add_trace(go.Pie(labels=data_regions['Region'], values=data_regions['indicateur_confiance_global'], name=""), 1, 1)
    fig.update_layout(
        title=f"Répartition de l'indicateur de confiance par région - année {upto_year}",
        title_x=0.5,
        width=800,
        height=800
    )
    fig.write_image(f"images/classement_regions_{upto_year}.png")


    data["Population 20-"] = data["Population Totale"] - data["Population 20+"]

    data["indicateur_confiance_individuel"] = calcul_indicateur_confiance(data)
    data["indicateur_confiance_individuel"] = round((data["indicateur_confiance_individuel"] - data["indicateur_confiance_individuel"].min()) / (data["indicateur_confiance_individuel"].max() - data["indicateur_confiance_individuel"].min()), 3)

    data["indicateur_confiance_global"] = round(data["indicateur_confiance_individuel"] * np.sqrt(data["Population 20+"] + 0.25 * data["Population 20-"]))
    data["indicateur_confiance_global"] = (data["indicateur_confiance_global"] - data["indicateur_confiance_global"].min()) / (data["indicateur_confiance_global"].max() - data["indicateur_confiance_global"].min())

    threshold_20pc = data["indicateur_confiance_global"].quantile(q=0.8)

    cl = data.loc[data["indicateur_confiance_global"]>=threshold_20pc, ["Country Name", "Region", "indicateur_confiance_global"]].sort_values(by="indicateur_confiance_global", ascending=False).reset_index(drop=True)
    cl["rang"] = cl.index + 1
    fig = make_subplots(rows = 1, cols = 1, vertical_spacing = 0.05, horizontal_spacing = 0.05, specs=[[{"type": "table"}]])
    fig.add_trace(go.Table(
            header=dict(
                values=['','<b>Pays</b>', '<b>Indice de confiance global</b>'],
                align=["left", "center"],
                line_color="darkslategray",
                fill=dict(color=['#cacaca', '#cacaca']),
            ), cells=dict(
                values=[cl["rang"], cl["Country Name"], cl["indicateur_confiance_global"]],
                line_color="darkslategray",
                fill=dict(color=['white', 'white']),
                align=["left", "center", "right"],
            ),
            columnwidth = [1,10, 10],

        ), 1, 1)
    fig.update_layout(
        title=f"Classement des indices de confiance par pays - année {upto_year}",
        title_x=0.5,
        margin={"t": 50},
        showlegend= False,
        height=800,
        width=800
    )
    fig.write_image(f"images/classement_{upto_year}.png")
    